In [1]:
import numpy as np
import sympy as smp
from sympy import *
from IPython.display import display, Latex

In [2]:
G, M, t, r, theta, phi, k, c = smp.symbols('G M t r θ Φ k c')
a = smp.Function('a')
a_dot = smp.Function('ȧ')

Gamma = IndexedBase("Gamma")
Riemann = IndexedBase("R")

In [3]:
Var = {
    t : 0,
    r : 1,
    theta : 2,
    phi : 3
}

In [4]:
Rav = {
    0 : t,
    1 : r,
    2 : theta,
    3 : phi
}

# Computing Riemann Tensor

### $$R^\alpha{}_{\beta\mu\nu}=\partial_\mu\Gamma^\alpha{}_{\nu\beta}-\partial_\nu\Gamma^\alpha{}_{\mu\beta}+\Gamma^\alpha{}_{\mu\lambda}\Gamma^\lambda{}_{\nu\beta}-\Gamma^\alpha{}_{\nu\lambda}\Gamma^\lambda{}_{\mu\beta}$$

In [5]:
# Compute Christoffel Symbols
def Christoffel_Symbols(m, i, j, Metric):
    inv_Metric = Metric.inv()
    C = 0
    for l in range(4):
        C += 1/2*inv_Metric[Var[m], l]*(smp.diff(Metric[Var[i],Var[m]], j)+smp.diff(Metric[Var[m],Var[j]], i)-smp.diff(Metric[Var[j],Var[i]], m))
    return simplify(C)

In [6]:
# Compute all Christoffel Symbols
def All_Christoffel_Symbols(Metric):
    All_C = np.zeros((4,4,4), dtype=object)
    for k in range(4):
        for i in range(4):
            for j in range(4):
                c = Christoffel_Symbols(Rav[k], Rav[i], Rav[j], Metric)
                if c != 0 :
                    All_C[k,i,j] = smp.nsimplify(c)
                    #display(Latex('$' + latex(Gamma[Rav[i],Rav[j]]**Rav[k]) + ' = ' + latex(c) + '$'))
    
    return All_C

In [7]:
# Compute Riemann Tensor component
def Rie(alpha, beta, mu, nu, Metric, All_C):
    G1 = 0
    G2 = 0
    for k in (t,r,theta,phi):
        All_C
        G1 += All_C[Var[alpha], Var[mu], Var[k]] * All_C[Var[k], Var[beta], Var[nu]]
        G2 += All_C[Var[alpha], Var[nu], Var[k]] * All_C[Var[k], Var[beta], Var[mu]]
        
    return simplify(smp.diff(All_C[Var[alpha], Var[beta], Var[nu]], mu) - smp.diff(All_C[Var[alpha], Var[beta], Var[mu]], nu) + G1 - G2)

In [8]:
# Compute all Riemann Tensor components
def All_r(Metric):
    All_C = All_Christoffel_Symbols(Metric)
    All_R = np.zeros((4,4,4,4), dtype=object)
    for i in range(4):
        for j in range(4):
            for l in range(4):
                for m in range(4):
                    R = Rie(Rav[i], Rav[j], Rav[l], Rav[m], Metric, All_C)
                    if R != 0 :
                        All_R[i,j,l,m] = R
                        display(Latex('$' + latex(Riemann[Rav[j],Rav[l],Rav[m]]**Rav[i]) + ' = ' + latex(R) + '$'))
    return All_R

### Minkowski Metric

In [9]:
# Convention (-+++)
Mink_Metric = np.array([[-1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
Mink = smp.Matrix(Mink_Metric)

In [10]:
Mink

Matrix([
[-1, 0, 0, 0],
[ 0, 1, 0, 0],
[ 0, 0, 1, 0],
[ 0, 0, 0, 1]])

In [14]:
All_R_Mink = All_r(Mink)

### Schwarschil Metric

In [14]:
# Convention (-+++)
S_Metric = np.zeros((4,4))
S = smp.Matrix(S_Metric)

S[0,0] = -(1-2*G*M/r)
S[1,1] = 1/(1-2*G*M/r)
S[2,2] = r**2
S[3,3] = r**2*smp.sin(theta)**2

S = smp.nsimplify(S)

In [15]:
S

Matrix([
[2*G*M/r - 1,                0,    0,              0],
[          0, 1/(-2*G*M/r + 1),    0,              0],
[          0,                0, r**2,              0],
[          0,                0,    0, r**2*sin(θ)**2]])

In [18]:
All_R_S = All_r(S)

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

### FLRW Metric

In [27]:
# Convention (+---)
FLRW_Metric = np.zeros((4,4))
FLRW = smp.Matrix(FLRW_Metric)

FLRW[0,0] = c**2
FLRW[1,1] = -a(t)**2/(1-k*r**2)
FLRW[2,2] = -a(t)**2*r**2
FLRW[3,3] = -a(t)**2*r**2*smp.sin(theta)**2

FLRW = smp.nsimplify(FLRW)

In [28]:
FLRW

Matrix([
[c**2,                      0,             0,                       0],
[   0, -a(t)**2/(-k*r**2 + 1),             0,                       0],
[   0,                      0, -r**2*a(t)**2,                       0],
[   0,                      0,             0, -r**2*a(t)**2*sin(θ)**2]])

In [30]:
All_R_FLRW = All_r(FLRW)

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

# Riemann Tensors Propreties

### $$R_{\alpha\beta\mu\nu} =  g_{\alpha\lambda}R^\lambda{}_{\beta\mu\nu}$$

In [52]:
alpha = theta
beta = phi
mu = phi
nu = theta

Metric = FLRW
All_C = All_Christoffel_Symbols(Metric)

### Antisymmetry in the last two indices

### $$R^{\alpha}{}_{\beta\mu\nu}=-R^{\alpha}{}_{\beta\nu\mu}$$

In [53]:
Rie(alpha, beta, mu, nu, Metric, All_C)

-r**2*(c**2*k + Derivative(a(t), t)**2)*sin(θ)**2/c**2

In [54]:
Rie(alpha, beta, nu, mu, Metric, All_C)

r**2*(c**2*k + Derivative(a(t), t)**2)*sin(θ)**2/c**2

### Antisymmetry in the first two indices

### $$R_{\alpha\beta\mu\nu}=-R_{\beta\alpha\mu\nu}$$

In [55]:
S[Var[alpha],Var[alpha]] * Rie(alpha, beta, mu, nu, Metric, All_C)

-r**4*(c**2*k + Derivative(a(t), t)**2)*sin(θ)**2/c**2

In [56]:
S[Var[beta],Var[beta]] * Rie(beta, alpha, mu, nu, Metric, All_C)

r**4*(c**2*k + Derivative(a(t), t)**2)*sin(θ)**2/c**2

### Symmetry under exchange of the two groups of indices (for Levi-Civita)

### $$R_{\alpha\beta\mu\nu}=R_{\mu\nu\alpha\beta}$$

In [60]:
S[Var[alpha],Var[alpha]] * Rie(alpha, beta, mu, nu, Metric, All_C)

-r**4*(c**2*k + Derivative(a(t), t)**2)*sin(θ)**2/c**2

In [59]:
S[Var[mu],Var[mu]] * Rie(mu, nu, alpha, beta, Metric, All_C)

r**4*(-c**2*k - Derivative(a(t), t)**2)*sin(θ)**2/c**2

### Bianchi identity

### $$R^{\alpha}{}_{\beta\mu\nu}+R^{\alpha}{}_{\mu\nu\beta}+R^{\alpha}{}_{\nu\beta\mu}=0$$

In [62]:
Rie(alpha, beta, mu, nu, Metric, All_C) + Rie(alpha, nu, beta, mu, Metric, All_C) + Rie(alpha, mu, nu, beta, Metric, All_C)

0